1. Install AWS CLI:
        ```
        $ curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
        $ sudo installer -pkg AWSCLIV2.pkg -target /
        ```
2. Get the SRR IDs by following the instructions [here](https://github.com/asadprodhan/A-Guide-to-Automatically-Downloading-NCBI-SRA-Reads?tab=readme-ov-file#step-2-collect-sra-accession-numbers)
